In [1]:
import sys
sys.path.append('../')
import pandas as pd
import random
import os
import numpy as np
import swifter

from cnt.model import (DesignEstimator, RelationExtractor, save_pipeline, load_pipeline, predict_re_single_sentence, 
relations_from_adjectives_df, relations_from_adjectives_single, concat_relations)
from cnt.annotate import (annotate, annotate_single_design, 
                          annotate_designs, 
                          extract_string_from_annotation, labeling_eng)
from cnt.extract_relation_single_verb import (path, NERTransformer, FeatureExtractor, RelationExtractor)
#from cnt.extract_relation import (path, NERTransformer, FeatureExtractor)
from cnt.evaluate import Metrics
from cnt.vectorize import (Doc2Str, Path2Str, Verbs2Str, AveragedPath2Vec, 
                           AveragedRest2Vec)
from cnt.io import (replace_left_right)
from cnt.io import  Database_Connection
from cnt.preprocess import Preprocess


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from itertools import product
import warnings
warnings.filterwarnings('ignore')

/home/bigdatalab/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Database access in text file: "mysql+mysqlconnector://root:YourConnection" -> Format user:password@IP/Database
f = open("/home/bigdatalab/Projects/D4N4/NLP_release_1.0/db_access.txt", "r")
access = f.read()
dc =  Database_Connection(access)

In [3]:
# Define all variables
id_col = "id" 
design_col = "design_en"
language = "_en"
add_columns = ["id", "name"+language, "alternativenames"+language, "class"]
id_col_RE = "design_id"
design_col_RE = "design_en"
ner_model_directory = "../cnt/trained_model/ner/english_new_single/" # for loading NER
ner_model_name = "english_cno"
re_model_directory = "../cnt/trained_model/re_subverb/"  # to save/load RE
re_model_name = "english_cno_new"


In [4]:
# optional - if already defined above
id_col = "id"
design_col = "design_en"
language = "_en"
add_columns = ["id", "name"+language, "alternativenames"+language, "class"]

In [5]:
# Get the annotated designs (ground truth) from the database
# ... under investigation
train = dc.create_own_query("""select design_id, 
(select design_en from nlp_training_designs as nlp where re.design_id=nlp.id) as design_en,
(select name_en from nlp_list_entities as ner where ner.id=re.subject) as s, 
(select class from nlp_list_entities as ner where ner.id=re.subject) as subject_class, 
(select name_en from nlp_list_entities as ner where ner.id=re.predicate) as p, 
(select name_en from nlp_list_entities as ner where ner.id=re.object) as o 
from nlp_relation_extraction_en_sub_verb as re;""")

In [6]:
train.head(5)

,design_id,design_en,s,subject_class,p,o
0,9,Amphora with ribbed surface and crooked handle...,amphora,OBJECT,holding,holding
1,10,"Bust of youthful Anchialos, right, wearing tae...",Anchialos,PERSON,wearing,wearing
2,24,"Bare-headed bust of Antoninus Pius, right, wea...",Antoninus Pius,PERSON,wearing,wearing
3,27,"Laureate bust of Antoninus Pius, right, wearin...",Antoninus Pius,PERSON,wearing,wearing
4,28,"Laureate bust of Antoninus Pius, right, wearin...",Antoninus Pius,PERSON,wearing,wearing


In [7]:
train.shape

(2067, 6)

In [8]:
# merging s, p, o to one triple into an array of length 1 for each entry - number of rows stay the same
train["y"] = train.apply(lambda row: [(row.s, row.subject_class, row.p, row.o)], axis=1)

In [9]:
train.head(2).style

,design_id,design_en,s,subject_class,p,o,y
0,9,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,amphora,OBJECT,holding,holding,"[('amphora', 'OBJECT', 'holding', 'holding')]"
1,10,"Bust of youthful Anchialos, right, wearing taenia. Border of dots.",Anchialos,PERSON,wearing,wearing,"[('Anchialos', 'PERSON', 'wearing', 'wearing')]"


In [10]:
# needed for the later merge
tmp = train.groupby("design_id").agg({"y": "sum"})

In [11]:
tmp.loc[tmp.index==1].style

,y
design_id,


In [12]:
X = train

In [13]:
# entries for one design are merged --> arrays contain all triples for each design
X = X.merge(tmp, left_on="design_id", right_on="design_id", suffixes=('', 'y'))

In [14]:
X = X[["design_id", "design_en", "yy"]].rename(columns={"yy":"y"})

In [15]:
X.head(5).style

,design_id,design_en,y
0,9,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,"[('amphora', 'OBJECT', 'holding', 'holding')]"
1,10,"Bust of youthful Anchialos, right, wearing taenia. Border of dots.","[('Anchialos', 'PERSON', 'wearing', 'wearing')]"
2,24,"Bare-headed bust of Antoninus Pius, right, wearing cuirass and paludamentum.","[('Antoninus Pius', 'PERSON', 'wearing', 'wearing')]"
3,27,"Laureate bust of Antoninus Pius, right, wearing cuirass and paludamentum.","[('Antoninus Pius', 'PERSON', 'wearing', 'wearing')]"
4,28,"Laureate bust of Antoninus Pius, right, wearing paludamentum.","[('Antoninus Pius', 'PERSON', 'wearing', 'wearing')]"


In [16]:
# duplicate entries for designs are removed (after merge the information is kept in a single row for each design)
X = X.drop_duplicates(['design_id'],keep="first")

In [17]:
# for preprocessing (preprocessing changes the original design text)
X["design_en_changed"] = ""

In [18]:
X.shape

(890, 4)

In [19]:
df_entities = dc.load_from_db("nlp_list_entities", add_columns)

In [20]:
# Add rules for preprocessing
preprocess = Preprocess()
preprocess.add_rule("horseman", "horse man")
preprocess.add_rule("horsemen", "horse men")

for index, row in df_entities.iterrows():
    if row["alternativenames_en"] is not None:
        if row["class"] != "VERB":
            standard_name = row["name_en"]
            alt_names = row["alternativenames_en"].split(", ")
            for alt_name in alt_names:
                preprocess.add_rule(alt_name, standard_name)

#### Bei römischen Zahlen scheint es noch kleine Probleme zu geben, daher nochmal extra manuell

In [21]:
for rule in list(preprocess.rules):
    if " I." in rule or " II." in rule or " III." in rule or " IV." in rule or " V." in rule:
        del preprocess.rules[rule]

In [22]:
for index, row in X.iterrows():
    if " I." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" I.", " I")
    if " II." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" II.", " II")
    if " III." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" III.", " III")
    if " IV." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" IV.", " IV")
    if " V." in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(" V.", " V")

In [23]:
# Deleting brackets and questionmarks

for index, row in X.iterrows():
    if "?" in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace("?", "")
    if "(" in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace("(", "")
    if ")" in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace(")", "")

## Apply Preprocessing

In [25]:
# Apply defined rules
X["design_en_changed"] = X.swifter.apply(lambda row: preprocess.preprocess_design(row.design_en, row.design_id)[0], axis=1)

Pandas Apply: 100%|███████████████████████████| 890/890 [00:17<00:00, 51.33it/s]


In [26]:
# Deleting brackets and questionmarks
X["design_en_changed"] = X.swifter.apply(lambda row: row["design_en_changed"].replace("?", "").replace("(", "").replace(")", ""), axis=1)

Pandas Apply: 100%|███████████████████████| 890/890 [00:00<00:00, 185459.59it/s]


In [27]:
X.rename(columns={"design_en":"design_en_orig", "design_en_changed":"design_en", "y":"annotations_orig"}, inplace=True)

In [28]:
# Mapping GT - prepocessing rules are also applied to GT
X["y"] = X.swifter.apply(lambda row: preprocess.preprocess_re(row["annotations_orig"], row.design_id), axis=1)

Pandas Apply: 100%|███████████████████████| 890/890 [00:00<00:00, 105405.35it/s]


### Train the RE model

In [29]:
# optional - if already defined above
id_col_RE = "design_id"
design_col_RE = "design_en"

In [30]:
classifier = LogisticRegression(max_iter=1000)
#classifier = RandomForestClassifier()
string_converter = Path2Str(pos=True) 
vectorizer = CountVectorizer(ngram_range=(1,3))
feature = make_pipeline(string_converter, vectorizer)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X[[id_col_RE, design_col_RE]], X[[id_col_RE, "y"]], test_size=0.25, random_state=33)

In [32]:
X.head()

,design_id,design_en_orig,annotations_orig,design_en,y
0,9,Amphora with ribbed surface and crooked handle...,"[(amphora, OBJECT, holding, holding)]",Amphora with ribbed surface and crooked handle...,"[(amphora, OBJECT, holding, holding)]"
1,10,"Bust of youthful Anchialos, right, wearing tae...","[(Anchialos, PERSON, wearing, wearing)]","Bust of youthful Anchialos, right, wearing tae...","[(Anchialos, PERSON, wearing, wearing)]"
2,24,"Bare-headed bust of Antoninus Pius, right, wea...","[(Antoninus Pius, PERSON, wearing, wearing)]","Bare-headed bust of Antoninus Pius, right, wea...","[(Antoninus Pius, PERSON, wearing, wearing)]"
3,27,"Laureate bust of Antoninus Pius, right, wearin...","[(Antoninus Pius, PERSON, wearing, wearing)]","Wreath bust of Antoninus Pius, right, wearing ...","[(Antoninus Pius, PERSON, wearing, wearing)]"
4,28,"Laureate bust of Antoninus Pius, right, wearin...","[(Antoninus Pius, PERSON, wearing, wearing)]","Wreath bust of Antoninus Pius, right, wearing ...","[(Antoninus Pius, PERSON, wearing, wearing)]"


In [33]:
X.shape

(890, 5)

In [34]:
X_train

,design_id,design_en
804,771,"Hound seated right; behind, ear of grain."
592,615,"Griffin lying left, right forepaw raised, with..."
730,708,"Head of Hermes, right, wearing wing petasus; i..."
1284,2256,"Horse prancing right; below, monogram."
805,24669,"Hygieia and Asclepius; to left, Hygieia standi..."
...,...,...
1513,5914,"Wreath bust of Macrinus, right; wearing scale ..."
1316,2302,"Nude bearded Heracles standing facing, head ri..."
1657,6081,"Athena standing facing, holding spear and shie..."
864,2394,"Head of Athena, right, wearing Attic helmet; w..."


In [35]:
y_train

,design_id,y
804,771,"[(hound, ANIMAL, seated_on, seated_on)]"
592,615,"[(griffin, ANIMAL, lying, lying), (griffin, AN..."
730,708,"[(Hermes, PERSON, wearing, wearing)]"
1284,2256,"[(horse, ANIMAL, prancing, prancing)]"
805,24669,"[(Hygieia, PERSON, standing, standing), (Hygie..."
...,...,...
1513,5914,"[(Macrinus, PERSON, wearing, wearing)]"
1316,2302,"[(Heracles, PERSON, standing, standing), (Hera..."
1657,6081,"[(Athena, PERSON, standing, standing), (Athena..."
864,2394,"[(Athena, PERSON, wearing, wearing)]"


#### load pretrained NER-Model

In [7]:
# optional - if already defined above
ner_model_directory = "../cnt/trained_model/ner/english_new_single/"
ner_model_name = "english_cno"

#### define RE-Model path

In [8]:
# optional - if already defined above
re_model_directory = "../cnt/trained_model/re_subverb/"
re_model_name = "english_cno"

In [38]:
# training
inner_pipeline = make_pipeline(feature, classifier)
pipeline = make_pipeline(NERTransformer(ner_model_directory, ner_model_name, id_col_RE, design_col_RE),
                         FeatureExtractor(ner_model_directory, ner_model_name, id_col_RE, design_col_RE),
                         RelationExtractor(inner_pipeline, re_model_directory, re_model_name, id_col_RE))
pipeline.fit(X_train, y_train)

NER(doc=Hound seated right; behind, ear of grain., subj=Hound, verb=seated)
NER(doc=Griffin lying left, right forepaw raised, with feather wing, tail under hindlegs and closed beak. Ground line., subj=Griffin, verb=lying)
NER(doc=Griffin lying left, right forepaw raised, with feather wing, tail under hindlegs and closed beak. Ground line., subj=feather, verb=lying)
NER(doc=Griffin lying left, right forepaw raised, with feather wing, tail under hindlegs and closed beak. Ground line., subj=wing, verb=lying)
NER(doc=Head of Hermes, right, wearing wing petasus; in front of him caduceus., subj=Head, verb=wearing)
NER(doc=Head of Hermes, right, wearing wing petasus; in front of him caduceus., subj=Hermes, verb=wearing)
NER(doc=Head of Hermes, right, wearing wing petasus; in front of him caduceus., subj=wing, verb=wearing)
NER(doc=Head of Hermes, right, wearing wing petasus; in front of him caduceus., subj=petasus, verb=wearing)
NER(doc=Head of Hermes, right, wearing wing petasus; in front of

NER(doc=Cybele throne left, wearing a polos, holding patera in outstretched right hand and pelleted scepter in left arm; under throne, lion seated, left., subj=hand, verb=seated)
NER(doc=Cybele throne left, wearing a polos, holding patera in outstretched right hand and pelleted scepter in left arm; under throne, lion seated, left., subj=scepter, verb=wearing)
NER(doc=Cybele throne left, wearing a polos, holding patera in outstretched right hand and pelleted scepter in left arm; under throne, lion seated, left., subj=scepter, verb=holding)
NER(doc=Cybele throne left, wearing a polos, holding patera in outstretched right hand and pelleted scepter in left arm; under throne, lion seated, left., subj=scepter, verb=seated)
NER(doc=Cybele throne left, wearing a polos, holding patera in outstretched right hand and pelleted scepter in left arm; under throne, lion seated, left., subj=arm, verb=wearing)
NER(doc=Cybele throne left, wearing a polos, holding patera in outstretched right hand and pel

NER(doc=Athena Nikephoros throne left, holding Nike in outstretched right hand; left arm resting on shield behind, decorated with gorgoneion; in background, transverse spear; in left field, monogram., subj=shield, verb=holding)
NER(doc=Athena Nikephoros throne left, holding Nike in outstretched right hand; left arm resting on shield behind, decorated with gorgoneion; in background, transverse spear; in left field, monogram., subj=shield, verb=resting)
NER(doc=Athena Nikephoros throne left, holding Nike in outstretched right hand; left arm resting on shield behind, decorated with gorgoneion; in background, transverse spear; in left field, monogram., subj=gorgoneion, verb=holding)
NER(doc=Athena Nikephoros throne left, holding Nike in outstretched right hand; left arm resting on shield behind, decorated with gorgoneion; in background, transverse spear; in left field, monogram., subj=gorgoneion, verb=resting)
NER(doc=Athena Nikephoros throne left, holding Nike in outstretched right hand; 

Pipeline(steps=[('nertransformer',
                 NERTransformer(design_col='design_en', id_col='design_id',
                                model_dir='../cnt/trained_model/ner/english_new_single/',
                                model_name='english_cno')),
                ('featureextractor',
                 FeatureExtractor(design_col='design_en', id_col='design_id',
                                  model_dir='../cnt/trained_model/ner/english_new_single/',
                                  model_name='english_cno')),
                ('relationextractor',
                 RelationExtractor(id_col='design_id', model_name='english_cno',
                                   output_dir='../cnt/trained_model/re_subverb/',
                                   pipeline=Pipeline(steps=[('pipeline',
                                                             Pipeline(steps=[('path2str',
                                                                              Path2Str(pos=True)),
       

## Save and Load model

In [39]:
save_pipeline(pipeline, re_model_directory, re_model_name)

In [9]:
model = load_pipeline(re_model_directory, re_model_name)

## Predict

In [ ]:
y_pred = model.predict(X_test)

In [42]:
metrics = Metrics()

In [43]:
precision, recall = metrics.score_precision_recall(y_test, y_pred)
F1 = (2*precision*recall) / (precision + recall)

In [44]:
print("Precision", round(precision*100,2))
print("Recall", round(recall*100,2))
print("F1", round(F1*100,2))

Precision 93.06
Recall 56.07
F1 69.97


## Map back

In [45]:
# results (that are based on the preprocessed designs) are mapped back to the original designs

X_test["y"] = y_test["y"]

In [46]:
# Deleting brackets and questionmarks
X_test["y_mapped"] = X_test.swifter.apply(lambda row: preprocess.map_re(row["y"], row.design_id), axis=1)

Pandas Apply: 100%|████████████████████████| 223/223 [00:00<00:00, 45439.65it/s]


In [47]:
X_test.head(5).style

,design_id,design_en,y,y_mapped
690,675,"Nude Heracles Farnese standing left, leaning on club with right hand set on rock, left hand on hip; lion skin over club.","[('Heracles', 'PERSON', 'standing', 'standing'), ('Heracles', 'PERSON', 'resting_on', 'resting_on')]","[('Heracles', 'PERSON', 'standing', 'standing'), ('Heracles', 'PERSON', 'resting_on', 'resting_on')]"
181,219,"Head of bearded Byzas, right, wearing crested helmet with open visor. Border of dots.","[('Byzas', 'PERSON', 'wearing', 'wearing')]","[('Byzas', 'PERSON', 'wearing', 'wearing')]"
1346,2323,"Nude Heracles sitting left on rock, his right hand outstretched, left resting on rock; at left, half-naked nymph standing facing, head right, holding a bow in both hand.","[('Heracles', 'PERSON', 'seated_on', 'seated_on'), ('Heracles', 'PERSON', 'resting_on', 'resting_on'), ('nymph', 'PERSON', 'standing', 'standing'), ('nymph', 'PERSON', 'holding', 'holding')]","[('Heracles', 'PERSON', 'seated_on', 'seated_on'), ('Heracles', 'PERSON', 'resting_on', 'resting_on'), ('nymph', 'PERSON', 'standing', 'standing'), ('nymph', 'PERSON', 'holding', 'holding')]"
692,676,"Nude Heracles standing facing, head left, resting right hand on club, left hand on hip, holding lion skin in left arm.","[('Heracles', 'PERSON', 'standing', 'standing'), ('Heracles', 'PERSON', 'resting_on', 'resting_on'), ('Heracles', 'PERSON', 'holding', 'holding')]","[('Heracles', 'PERSON', 'standing', 'standing'), ('Heracles', 'PERSON', 'resting_on', 'resting_on'), ('Heracles', 'PERSON', 'holding', 'holding')]"
47,24726,"Nude Apollo standing left, holding patera in outstretched right hand over altar and laurel branch in left hand.","[('Apollo', 'PERSON', 'standing', 'standing'), ('Apollo', 'PERSON', 'holding', 'holding')]","[('Apollo', 'PERSON', 'standing', 'standing'), ('Apollo', 'PERSON', 'holding', 'holding')]"


## Auto relations

In [55]:
# adding relations based on adjectives - experimental
obj_list = {
"veiled": ("wearing", "Veil", "before"),
"draped": ("wearing", "Clothing", "before"),
"helmeted": ("wearing", "Helmet", "before"),
"diademed": ("wearing", "Diadem", "before"),
"turreted": ("wearing", "Mural crown", "before"),
"enthroned": ("seated_on", "Throne", "after"),

}

In [56]:
y_pred["design_en"] = X_test.design_en

In [57]:
y_pred = relations_from_adjectives_df(y_pred, "design_en", "y", ner_model_directory, ner_model_name, id_col, design_col, obj_list, entities_to_consider=["PERSON"])

In [58]:
y_pred

,design_id,y,design_en
690,675,[],"Nude Heracles Farnese standing left, leaning o..."
181,219,[],"Head of bearded Byzas, right, wearing crested ..."
1346,2323,[],"Nude Heracles sitting left on rock, his right ..."
692,676,[],"Nude Heracles standing facing, head left, rest..."
47,24726,[],"Nude Apollo standing left, holding patera in o..."
...,...,...,...
1845,6250,[],"Artemis standing facing, holding torch in both..."
1362,2332,[],"Griffin seated left, right forepaw raised, wit..."
134,144,"[(Athena, PERSON, throne, throne)]","Athena Nikephoros throne left, holding Nike in..."
192,236,[],"Capricorn standing right; above, cornucopia. B..."


In [49]:
# for testing a single design

design = "Diademed Athena to the left and helmeted Ares to the right, holding sword."
# rule based - see above 
auto_relations = relations_from_adjectives_single(design,ner_model_directory, ner_model_name, id_col_RE, design_col_RE, obj_list)
# results of the model
model_relations = predict_re_single_sentence(model, design, id_col_RE, design_col_RE)
# merging both results
concat_relations(auto_relations, model_relations)

[('Ares', 'PERSON', 'wearing', 'Helmet', 'OBJECT'),
 ('Athena', 'PERSON', 'wearing', 'Diadem', 'OBJECT'),
 ('Athena', 'PERSON', 'holding', 'sword', 'OBJECT'),
 ('Ares', 'PERSON', 'holding', 'sword', 'OBJECT')]

In [35]:
# optional -only if results need to be uploaded to database
# preprocessing
upload = True
f = open("/home/bigdatalab/Projects/D4N4/NLP_release_1.0/db_access.txt", "r")
access = f.read()
dc =  Database_Connection(access)
cnt_designs = dc.load_designs_from_db("data_designs", [id_col, design_col])
cnt_designs = cnt_designs.rename(columns={"id": "design_id"})

cnt_designs["design_en_changed"] = cnt_designs.swifter.apply(lambda row: preprocess.preprocess_design(row.design_en, row.design_id)[0], axis=1)
cnt_designs["design_en_changed"] = cnt_designs.swifter.apply(lambda row: row["design_en_changed"].replace("?", "").replace("(", "").replace(")", ""), axis=1)

Pandas Apply: 100%|█████████████████████| 7759/7759 [00:00<00:00, 207181.18it/s]


In [36]:
# optional -only if results need to be uploaded to database
cnt_designs = cnt_designs.rename(columns={"design_en": "design_en_org"})
cnt_designs = cnt_designs.rename(columns={"design_en_changed": "design_en"})

In [37]:
# optional -only if results need to be uploaded to database - TODO: check by Sebastian on BDL-Server
upload = True
if upload ==True:
    cnt_pred = model.predict(cnt_designs, rdf=True)
    cnt_pipeline_output = pd.DataFrame([(str(designid), *relation) for  _, (designid, relation_list) in cnt_pred.iterrows()
                    for relation in relation_list],
            columns=["DesignID", "Person", "Label_Person", "Relation", "Object"])
    cnt_pipeline_output.to_sql("cnt_pipeline_output_v3",dc.mysql_connection,if_exists="replace", index=False)

In [59]:
cnt_designs.head()

,design_id,design_en_org,design_en
0,1,Diademed head of deified Alexander the Great w...,Diadem head of deified Alexander the Great wit...
1,6,"Altar, lighted and garlanded.","Altar, lighted and garland."
2,8,Prize amphora on ornamental stand; within line...,Prize amphora on ornamental stand; within line...
3,9,Amphora with ribbed surface and crooked handle...,Amphora with ribbed surface and crooked handle...
4,10,"Bust of youthful Anchialos, right, wearing tae...","Bust of youthful Anchialos, right, wearing tae..."


In [54]:
cnt_pipeline_output = pd.DataFrame([(str(designid), *relation) for  _, (designid, relation_list) in cnt_pred.iterrows()
                    for relation in relation_list], columns=["DesignID", "Person", "Label_Person", "Relation", "Object"])
cnt_pipeline_output.to_sql("cnt_pipeline_output_v3",dc.mysql_connection,if_exists="replace", index=False)

9119